In [1]:
import pandas as pd
import numpy as np
import torch
import torchaudio
import textgrids
import torchaudio.transforms as T

SAMPLE_RATE = 16000

/work/dpandya/miniconda3/envs/learning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
df = pd.read_csv('VADKaggle.csv')
df.loc[0][1]

'../audioData/VAD_Kaggle/Data/Annotation/Female/PTDB-TUG/mic_F01_sa2.TextGrid'

In [56]:
#df['audio_files'], df['annotation_files']

class AudioUtils():

    def resample_audio(sig, org_sr):
        resampler = T.Resample(org_sr, SAMPLE_RATE)
        resampled_sig = resampler(sig)
        return resampled_sig
    
    def open(aud_fn):
        sig, sr = torchaudio.load(aud_fn)
        duration = sig.shape[1]/sr
        #print(sig.shape)
        if sr != SAMPLE_RATE:
            sig = AudioUtils.resample_audio(sig, sr)
        
        if sig.shape[1]%SAMPLE_RATE == 0:
            duration = sig.shape[1]/SAMPLE_RATE
        else:
            sig_np = sig.squeeze().numpy()
            #print(sig_np.shape)
            en = (int)((np.round(duration)+1)*SAMPLE_RATE - sig_np.shape[0])
            sig_np = np.pad(sig_np, (0,en), mode = 'constant')
            #sig = torch.from_numpy(sig_np).unsqueeze(0)
            duration = sig_np.shape[0] / SAMPLE_RATE
        
        return sig_np, SAMPLE_RATE, duration
    
    def get_second_wise_mfcc(signal, duration):
        mfcc_list = []
        MFCC = T.MFCC(sample_rate=SAMPLE_RATE, n_mfcc=13)
        for i in range(0,(int)(duration)):
            mfcc_list.append(MFCC(signal[0][i:(i+SAMPLE_RATE)].unsqueeze(0)).squeeze())

        return mfcc_list

In [6]:
class AnnotUtils():

    def get_speech_secs(fname):
        grid = textgrids.TextGrid(fname)
        speech_secs = []
        for i in grid['silences']:
            if i.text == '1':
                #speech_secs.append([(np.round(i.xmin, decimals=2)), (np.round(i.xmax, decimals=2))])
                speech_secs.append([(int)(np.round(i.xmin, decimals=2)), (int)(np.round(i.xmax, decimals=2))])
        return speech_secs
    

    def get_labs_for_secs(speech_secs, duration):
        
        labels = [i*0 for i in range(0, (int)(duration))]

        for i in range(0,len(speech_secs)):

            if speech_secs[i][0] == speech_secs[i][1]:
                labels[speech_secs[i][0]-1] = 1
            else:
                for j in range(speech_secs[i][0], speech_secs[i][1]):
                    
                    labels[j-1] = 1
                    labels[j] = 1

        return labels 
        


In [4]:
# Remember to change the sig dtype to tensor and unsqueeze
sig, sr, dur = AudioUtils.open('../audioData/VAD_Kaggle/Data/Audio/Female/PTDB-TUG/mic_F01_si528.wav')
sample_ann = '../audioData/VAD_Kaggle/Data/Annotation/Female/PTDB-TUG/mic_F01_si528.TextGrid'
ann_list = AnnotUtils.get_labs_for_secs(AnnotUtils.get_speech_secs(sample_ann),dur)
#mfcc_list = AudioUtils.get_second_wise_mfcc(sig, dur)
print(dur, ann_list)

4
5
6
11.0 [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]


In [24]:
aud_files_secWise = []
labels_list = []

for i,row in df.iterrows():
    
    aud_fname = df['audio_files'][i]
    annot_fname = df['annotation_files'][i]
    #print(aud_fname.split('.wav')[0])
    sig, sr, dur = AudioUtils.open(aud_fn=aud_fname)
    
    lll = AnnotUtils.get_labs_for_secs(AnnotUtils.get_speech_secs(annot_fname), dur)
    print(len(lll), dur)
    for j in range(0,(int)(dur)):
        aud_files_secWise.append(aud_fname.split('.wav')[0]+'_'+ (str)(j))
        labels_list.append(lll[j])
        

8 8.0
10 10.0
8 8.0
8 8.0
10 10.0
11 11.0
9 9.0
9 9.0
10 10.0
11 11.0
7 7.0
8 8.0
9 9.0
11 11.0
9 9.0
9 9.0
10 10.0
10 10.0
10 10.0
8 8.0
9 9.0
10 10.0
8 8.0
8 8.0
11 11.0
11 11.0
9 9.0
11 11.0
11 11.0
8 8.0
8 8.0
10 10.0
12 12.0
11 11.0
7 7.0
9 9.0
8 8.0
8 8.0
7 7.0
10 10.0
7 7.0
7 7.0
7 7.0
8 8.0
9 9.0
10 10.0
8 8.0
8 8.0
10 10.0
10 10.0
8 8.0
7 7.0
10 10.0
10 10.0
9 9.0
10 10.0
9 9.0
7 7.0
7 7.0
9 9.0
8 8.0
8 8.0
10 10.0
10 10.0
9 9.0
9 9.0
7 7.0
7 7.0
6 6.0
8 8.0
7 7.0
8 8.0
10 10.0
6 6.0
6 6.0
7 7.0
7 7.0
7 7.0
8 8.0
6 6.0
8 8.0
6 6.0
7 7.0
7 7.0
8 8.0
10 10.0
8 8.0
9 9.0
7 7.0
9 9.0
9 9.0
8 8.0
6 6.0
7 7.0
7 7.0
9 9.0
8 8.0
9 9.0
8 8.0
10 10.0
10 10.0
11 11.0
7 7.0
9 9.0
7 7.0
8 8.0
8 8.0
9 9.0
8 8.0
6 6.0
10 10.0
9 9.0
7 7.0
8 8.0
7 7.0
9 9.0
7 7.0
8 8.0
7 7.0
8 8.0
4 4.0
3 3.0
4 4.0
4 4.0
5 5.0
4 4.0
4 4.0
4 4.0
6 6.0
6 6.0
3 3.0
4 4.0
4 4.0
7 7.0
3 3.0
6 6.0
6 6.0
3 3.0
6 6.0
4 4.0
5 5.0
6 6.0
6 6.0
4 4.0
4 4.0
3 3.0
4 4.0
3 3.0
2 2.0
3 3.0
4 4.0
4 4.0
4 4.0
4 4.0
5 5.0
5 5.0


In [22]:
df2 = pd.DataFrame(columns=['audio_list_secs', 'secWise_labs'])
df2['audio_list_secs'] = aud_files_secWise
df2['secWise_labs'] = labels_list

df2.head()

,audio_list_secs,secWise_labs
0,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,0
1,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,0
2,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,1
3,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,1
4,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,1


In [23]:
df2.to_csv('secWise_labs.csv', index=False)

In [67]:
#df2.loc[0][0][:-2]
type(df2.loc[46][1])

numpy.int64

In [5]:
mfcc_list = AudioUtils.get_second_wise_mfcc(sig, dur)
mfcc_list[7].shape, dur

/work/dpandya/miniconda3/envs/learning/lib/python3.10/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


(torch.Size([13, 81]), 11.0)

In [78]:
from torch.utils.data import DataLoader, Dataset, random_split

class VAD_Dataset(Dataset):
    def __init__(self, df) -> None:
        #super().__init__()
        self.df = df
        self.sr = 16000

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):

        aud_part = self.df.loc[index][0]
        labels = self.df.loc[index][1]
        
        if aud_part[-2] == '_':
            n_sec = (int)(aud_part[-1])
            aud_path = aud_part[:-2]+'.wav'
        elif aud_part[-3] == '_':
            n_sec = (int)(aud_part[-2:])
            aud_path = aud_part[:-3]+'.wav'
        print(n_sec)
        sig, sam, dur = AudioUtils.open(aud_path)
        sig = torch.from_numpy(sig[n_sec*SAMPLE_RATE:((n_sec+1)*SAMPLE_RATE)]).unsqueeze(0)

        mfcc_list = AudioUtils.get_second_wise_mfcc(sig, dur)
        #labels = AnnotUtils.get_labs_for_secs(AnnotUtils.get_speech_secs(annot_path), dur)

        return len(mfcc_list), labels

In [83]:
myds = VAD_Dataset(df=df2[0:10])

In [84]:
for i in myds:
    print(i)

0
(8, 0)
1
(8, 0)
2
(8, 1)
3
(8, 1)
4
(8, 1)
5
(8, 0)
6
(8, 0)
7
(8, 0)
0
(10, 0)
1
(10, 0)


KeyError: 10

In [85]:
from IPython.display import Audio, display

display(Audio(sig.squeeze().numpy(), rate=sr))
#print(sig[0].unique())